In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import json
import spacy as sp
import random

nlp = sp.load("es_core_news_md")

In [2]:
df = pd.read_csv("Data/last-cook.csv")
df_cook = pd.read_csv("Data/intents_cook.csv")
df_cook_others = pd.read_csv('Data/intents_cook_others.csv')

print("Processing the Intents.....")
with open('Data/intents.json') as json_data:
    intents = json.load(json_data)

df_cook_others["label"] = df_cook_others.label.map({ 'cook': 1, 'others': 0})
df_cook["label"] = df_cook.label.map({ 'name_to_prepare': 1, 'ingredient_to_name': 0})
df_cook

Processing the Intents.....


,Unnamed: 0,text,label
0,0,¿Cómo se hace la receta de las galletas de avena?,1
1,1,¿Cuál es la mejor manera de cocinar una pizza ...,1
2,2,¿Cómo puedo hacer salsa de tomate casera?,1
3,3,¿Tienes una buena receta para hacer sopa de po...,1
4,4,¿Cómo se hace la masa para los tacos mexicanos?,1
...,...,...,...
105,105,¿Cuál es el nombre de esa receta que lleva arr...,0
106,106,¿Cómo se llama esa receta que lleva espaguetis...,0
107,107,¿Cuál es el nombre de esa receta que lleva sal...,0
108,108,"¿Cómo se llama esa receta que lleva pollo, jen...",0


In [3]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# print('Accuracy score: ', format(accuracy_score(y_test, predictions)))

# print('Precision score: ', format(precision_score(y_test, predictions)))
# print('Recall score: ', format(recall_score(y_test, predictions)))
# print('F1 score: ', format(f1_score(y_test, predictions)))

### clasificar si me estas hablando de cocina o no

In [4]:
def cook_others(text):
    # Dividir el conjunto de datos de entrenamiento y test
    X_train, X_test, y_train, y_test = train_test_split(df_cook_others['text'], df_cook_others['label'], random_state=1)
    
    count_vector = CountVectorizer()
    
    training_data = count_vector.fit_transform(X_train) 
    
    testing_data = count_vector.transform(X_test)
    text_data = count_vector.transform([text])

    naive_bayes = MultinomialNB()
    naive_bayes.fit(training_data, y_train)
    
    return naive_bayes.predict(text_data)

### clasificar que tipo de pregunta de cocina

In [5]:
def cook(text):
    # Dividir el conjunto de datos de entrenamiento y test
    X_train, X_test, y_train, y_test = train_test_split(df_cook['text'], df_cook['label'], random_state=1)
    
    count_vector = CountVectorizer()
    
    training_data = count_vector.fit_transform(X_train)
    
    testing_data = count_vector.transform(X_test)
    text_data = count_vector.transform([text])

    naive_bayes = MultinomialNB()
    naive_bayes.fit(training_data, y_train)
    
    return naive_bayes.predict(text_data)

### Clasificar en que tipo de pregunta varia

In [6]:
def others(text):
    x = []
    y_train = []
    for i in range(4):
    #     print(intents["intents"][i])
        label = intents["intents"][i]["tag"]
        for j in intents["intents"][i]["patterns"]:
            x.append(j)
            y_train.append(label)
            
    # Instantiate the CountVectorizer method
    count_vector = CountVectorizer()
    # Fit the training data and then return the matrix
    training_data = count_vector.fit_transform(x)
    text_data = count_vector.transform([text])

    naive_bayes = MultinomialNB()
    naive_bayes.fit(training_data, y_train)
    
    return naive_bayes.predict(text_data)

### Modelos que devuelven la respuesta

In [7]:
# devolver una respuesta de cocina segun el intent
def search(text, first, second):
    ma = -1000000
    sol = ""
    doc = nlp(text)
    for i in range(df.shape[0]): 
        aux = nlp(str(df[first][i]))
        value = doc.similarity(aux)
        if (value > ma):
            ma = value
            sol = df[second][i]
    return sol
  
# Devolver una respuesta varia aleatoria segun el intent
def response_others(tag):
    for i in range(4):
    #     print(intents["intents"][i])
        label = intents["intents"][i]["tag"]
        if label == tag:
            rand = int(random.uniform(0,len(intents["intents"][i]["responses"])))
            print(rand)
            return intents["intents"][i]["responses"][rand]

### Obtener la respuesta

In [8]:
def get_response(text):
    firstIntent = cook_others(text)[0]
    if firstIntent == 0:
        intentOthers = others(text)[0]
        return response_others(intentOthers)
    else:
        cook_intents = cook(text) 
        if cook_intents == 0:
            return search(text, "ingredients", "title")

        if cook_intents == 1:
            return search(text, "title", "instructions")
    return firstIntent

get_response("¿Cuál es el nombre de esa receta que lleva camarones, arroz y especias?")

'Pescado en lámina con cebollas dulces, tomates y mojo verde '